In [1]:
## Optional
# Format output of Jupyter Notebook
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Hide python warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import isnull, when, count, col, regexp_extract

# TODO - parameterize endpoints/ports such as minio:9000 with envs

# Define config for SparkSession,
# such as Iceberg catalog that utilizes minio, an S3-compatible local object storage
spark_configs = {
    'spark.master': 'spark://spark-iceberg:7077',
    'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkCatalog',
    # 'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkSessionCatalog',
    'spark.sql.catalog.airline.io-impl': 'org.apache.iceberg.aws.s3.S3FileIO',
    'spark.sql.catalog.airline.s3.endpoint': 'http://minio:9000',
    'spark.sql.catalog.airline.type': 'rest',
    'spark.sql.catalog.airline.uri': 'http://rest:8181',
    'spark.sql.catalog.airline.warehouse': 's3://warehouse',
    'spark.sql.defaultCatalog': 'airline'
}

# Initialize SparkSession
spark = (
    SparkSession
    .builder
    .appName('EDA')
    .config(map=spark_configs)
    .getOrCreate()
)

spark.conf



filename = '/home/iceberg/data/flights.csv'

24/12/19 19:55:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
import pprint as pp

# print(spark.sparkContext.uiWebUrl)

print(spark.conf.get('spark.app.id'))
print(spark.conf.get('spark.app.name'))
print(spark.conf.get('spark.app.startTime'))
print(spark.conf.get('spark.master'))
print()
print(spark.conf.get('spark.sql.catalog.airline'))
print(spark.conf.get('spark.sql.catalog.airline.io-impl'))
print(spark.conf.get('spark.sql.catalog.airline.s3.endpoint'))
print(spark.conf.get('spark.sql.catalog.airline.type'))
print(spark.conf.get('spark.sql.catalog.airline.uri'))
print(spark.conf.get('spark.sql.catalog.airline.warehouse'))
print()
                     
# pp.pp(dict(sc.getConf().getAll()), sort_dicts=True, )

local-1734638114350
EDA
1734638113495
spark://spark-iceberg:7077

org.apache.iceberg.spark.SparkCatalog
org.apache.iceberg.aws.s3.S3FileIO
http://minio:9000
rest
http://rest:8181
s3://warehouse



# Ingest raw csv data

In [37]:
# Check for alphabetic characters in flight_number, this is useless...
raw_data = spark.read.option('header', True).csv(filename)
raw_data.where(regexp_extract('flight_number', '^\d+$', 0) == '').show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [8]:
# TODO: Fix nullables column, cant set column as Not-Nullable for some reason...

# Define schema
flights_schema = StructType([
    StructField("year", IntegerType(), False),   # Non-Nullable
    StructField("month", IntegerType(), False),  # Non-Nullable
    StructField("day", IntegerType(), False),    # Non-Nullable
    StructField("day_of_week", IntegerType(), False),    # Non-Nullable
    StructField("airline", StringType(), False),         # Non-Nullable
    StructField("flight_number", IntegerType(), False),  # Non-Nullable
    StructField("tail_number", StringType(), True),
    StructField("origin_airport", StringType(), False),  # Non-Nullable
    StructField("destination_airport", StringType(), False),   # Non-Nullable
    StructField("scheduled_departure", IntegerType(), False),  # Non-Nullable
    StructField("departure_time", IntegerType(), True),
    StructField("departure_delay", IntegerType(), True),
    StructField("taxi_out", IntegerType(), True),
    StructField("wheels_off", IntegerType(), True),
    StructField("scheduled_time", IntegerType(), True),
    StructField("elapsed_time", IntegerType(), True),
    StructField("air_time", IntegerType(), True),
    StructField("distance", IntegerType(), False),  # Non-Nullable
    StructField("wheels_on", IntegerType(), True),
    StructField("taxi_in", IntegerType(), True),
    StructField("scheduled_arrival", IntegerType(), False),  # Non-Nullable
    StructField("arrival_time", IntegerType(), True),
    StructField("arrival_delay", IntegerType(), True),
    StructField("diverted", IntegerType(), False),   # Non-Nullable
    StructField("cancelled", IntegerType(), False),  # Non-Nullable
    StructField("cancellation_reason", StringType(), True),
    StructField("air_system_delay", IntegerType(), True),
    StructField("security_delay", IntegerType(), True),
    StructField("airline_delay", IntegerType(), True),
    StructField("late_aircraft_delay", IntegerType(), True),
    StructField("weather_delay", IntegerType(), True)
])

# Read flights.csv
flights_df = spark.read.schema(flights_schema).csv(
    '/home/iceberg/data/flights.csv',
    schema=flights_schema,
    enforceSchema=True,
    header = True
).cache()

flights_df.show()
flights_df.printSchema()
flights_df.explain()

24/12/19 20:20:22 WARN CacheManager: Asked to cache already cached data.


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|year|month|day|day_of_week|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [ ]:
flights.filter('year')

# Explore data!
TODO:
- Check statistics of important columns, like what are the percentiles for departure/arrival delay?
- Check max values for delay

## Total number of flight records: 5,819,079

In [32]:
flights_df.count()

5819079

## Total number of cancelled flights: 89,884, Total number of diverted flights: 15,187
These will cause nulls in other columns.

In [37]:
flights_df.where('cancelled = 1').count()

89884

In [35]:
flights_df.where('diverted = 1').count()

15187

## Get Null counts for each column

In [31]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

## Investigate some causes for nulls in each column

In [48]:
# List of columns with null values
null_columns = [col for col in flights_df.columns if flights_df.filter(f"{col} IS NULL").count() > 0]

# Iterate through columns to investigate null causes
for col in null_columns:
    print(f"Investigating nulls in column: {col}")
    # Count rows where column is null and cancelled == 1
    count = flights_df.where(f"`{col}` IS NULL AND cancelled = 1").count()
    print(f"Nulls in {col} where flight is cancelled: {count}\n")

Investigating nulls in column: tail_number


Nulls in tail_number where flight is cancelled: 14721

Investigating nulls in column: departure_time


Nulls in departure_time where flight is cancelled: 86153

Investigating nulls in column: departure_delay


Nulls in departure_delay where flight is cancelled: 86153

Investigating nulls in column: taxi_out


Nulls in taxi_out where flight is cancelled: 89047

Investigating nulls in column: wheels_off


...

Nulls in air_system_delay where flight is cancelled: 89884

Investigating nulls in column: security_delay


Nulls in security_delay where flight is cancelled: 89884

Investigating nulls in column: airline_delay


Nulls in airline_delay where flight is cancelled: 89884

Investigating nulls in column: late_aircraft_delay


Nulls in late_aircraft_delay where flight is cancelled: 89884

Investigating nulls in column: weather_delay


Nulls in weather_delay where flight is cancelled: 89884



#### Tail_number - all Nulls are from cancelled flights
Although, not all cancelled flights have Nulls in tail_number. This might be explained by flights that are cancelled well in advance.
**TODO: Check cancellation reasons**

In [40]:
flights_df.where('tail_number IS NULL AND cancelled = 1').count()

14721

In [44]:
flights_df.where('departure_time IS NULL and cancelled = 1').count()

86153

#### Scheduled_time - 6 Nulls, 5 out of 6 flight were cancelled, 1 flight was diverted
Scheduled_time is the estimated time in air?

In [28]:
flights_df.where('scheduled_time IS NULL').show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|year|month|day|day_of_week|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

# Write to staging Iceberg Table

In [26]:
# Drop flights table
spark.sql('DROP TABLE airline.db.flights PURGE')

DataFrame[]

In [27]:
# Create flights table if not exists
spark.sql("""
CREATE TABLE IF NOT EXISTS airline.db.flights (
    year INT,
    month INT,
    day INT,
    day_of_week INT,
    airline STRING,
    flight_number INT,
    tail_number STRING,
    origin_airport STRING,
    destination_airport STRING,
    scheduled_departure INT,
    departure_time INT,
    departure_delay INT,
    taxi_out INT,
    wheels_off INT,
    scheduled_time INT,
    elapsed_time INT,
    air_time INT,
    distance INT,
    wheels_on INT,
    taxi_in INT,
    scheduled_arrival INT,
    arrival_time INT,
    arrival_delay INT,
    diverted INT,
    cancelled INT,
    cancellation_reason STRING,
    air_system_delay INT,
    security_delay INT,
    airline_delay INT,
    late_aircraft_delay INT,
    weather_delay INT
)
USING iceberg
PARTITIONED BY (month)
""")

DataFrame[]

In [24]:
# # Create database if not exists
# spark.sql('CREATE DATABASE IF NOT EXISTS db')

# print(spark.catalog.tableExists('airline.db.flights'))

# # Create flights table if not exists
# if not spark.catalog.tableExists('airline.db.flights'):
#     spark.catalog.createTable(
#         'airline.db.flights',
#         schema=flights_schema,
#         source='iceberg',
#         partitioning='month'
#     )

# spark.catalog.getTable('airline.db.flights')

False


Table(name='flights', catalog='airline', namespace=['db'], description=None, tableType='MANAGED', isTemporary=False)

In [28]:
# Write flights data
flights_df \
    .writeTo('airline.db.flights') \
    .append()
    # .sort('day_of_week') \

In [29]:
spark.sql('DESCRIBE EXTENDED airline.db.flights').show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|               year|      int|   NULL|
|              month|      int|   NULL|
|                day|      int|   NULL|
|        day_of_week|      int|   NULL|
|            airline|   string|   NULL|
|      flight_number|      int|   NULL|
|        tail_number|   string|   NULL|
|     origin_airport|   string|   NULL|
|destination_airport|   string|   NULL|
|scheduled_departure|      int|   NULL|
|     departure_time|      int|   NULL|
|    departure_delay|      int|   NULL|
|           taxi_out|      int|   NULL|
|         wheels_off|      int|   NULL|
|     scheduled_time|      int|   NULL|
|       elapsed_time|      int|   NULL|
|           air_time|      int|   NULL|
|           distance|      int|   NULL|
|          wheels_on|      int|   NULL|
|            taxi_in|      int|   NULL|
+-------------------+---------+-------+
only showing top 20 rows



#### HIDDEN PARTITIONING!?
When partitioning on year, still getting 12 files, but still 1 partition.

Dataset came sorted by date, so hidden partitioning was automatically splitting into 12 parquet files?

When partitioning on month, what is the behavior?

In [24]:
metrics = spark.sql('select readable_metrics from airline.db.flights.files').collect()

for metric in metrics:
    temp = metric.asDict()['readable_metrics'].asDict()['month']
    print(temp)

Row(column_size=2658, value_count=865543, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=12)
Row(column_size=2585, value_count=844600, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=12)
Row(column_size=2603, value_count=855897, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=12)
Row(column_size=2647, value_count=872521, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=12)
Row(column_size=2643, value_count=862209, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=12)
Row(column_size=2128, value_count=700545, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=12)
Row(column_size=2456, value_count=817764, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=12)


#### Checking table metadata

In [29]:
%%sql
SELECT
    *
FROM airline.db.flights.partitions

record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
5819079,7,132410345,0,0,0,0,2024-12-19 05:27:21.064000,2043671014194954771


In [28]:
%%sql
SELECT
    *
FROM airline.db.flights.files

+---------+----------------------------------------------------------------------------------------------+-------------+---------+--------------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+---------------+--------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
%%sql
SELECT
    SUM(file_size_in_bytes)
FROM airline.db.flights.files

sum(file_size_in_bytes)
132410345


In [26]:
%%sql
SELECT
    file_size_in_bytes
FROM airline.db.flights.files

file_size_in_bytes
19788655
19204359
19411983
19891606
19622735
15925957
18565050
